### Initialization

In [5]:
import numpy as np
import pandas as pd
import sys
import gams.core.numpy
import gamspy as gp
from global_container import container, Set, Variable, Parameter
from collections import Counter

### We're going to read data to a .gdx file!

### GAMSPy set-up + introduction

In [ ]:
m=gp.Container()
#product=gp.Set(m,name="product",description="energy product type, e.g. coal, electricity, etc.")
#purpose=gp.Set(m,name="purpose",description="Purpose of energy usage, e.g. heating, transport, etc.")
#indu=gp.Set(m,name="indu",description="Industry, energy sectors,c and tax. Tax is from io, maybe split idk")
#tax=gp.Set(m,name="tax",description="tax type, e.g. CO2 tax, etc.")
#item=gp.Set(m,name="item",description="emissions differing by accounting method")

'''
UPDATE THIS: Fix this intro


The data we read in this file is from several different .xlsx sheets. I always read all the data from one spreadsheet before starting another, and this is always indicated by a header named filename.xlsx.
There may be several chunks of code dedicated to a single spreadsheet depending on the side. Unless there is a header named filename.xlsx, the chunk of code treats data from the same spreadsheet as the previous chunk.
The packages I use are all imported in the chunk immediately above this one. Longer comments, or comments that I think contain information that may be relevant to the reader is commented out like here, whereas
comments that are mainly for the sake of structure and do not contain new or noteworthy information are just hashtagged out.
As of 20/2/25 this script is unrevised by others. 
The script runs reasonably efficiently with a few exceptions, it does however rely quite heavily on the pandas-package atm.
In terms of robustness, I rely quite heavily on the naming conventions and datastructure in the datasheets.
The names are not extracted, but typed which comes with some risk should those conventions change.
The gp.Set(), gp.Parameter()-functions that I rely on for datatransfer are also snesitive to changes in the data-structure, that is domain-names must be entered in the "correct" order, especially when relying on the
very efficient domain_forwarding for inhabiting sets.
Sets and variables are named deliberately to be as unequivocal as possible regarding what input-data from the spreadsheets they contain. 
This means that many objects have rather long, tongue-twisting names. The general naming convention is spreadsheet_somesubset_somesubsetofsubset such that ideally, the output generated at the end of this script,
can be traced back to a cell(s) in the spreadsheet by anyone pedantic enough to want to do so without having to inspect the code.
'''

'\nUPDATE THIS:\nThe data we read in this file is from several different .xlsx sheets. I always read all the data from one spreadsheet before starting another, and this is always indicated by a header named filename.xlsx.\nThere may be several chunks of code dedicated to a single spreadsheet depending on the side. Unless there is a header named filename.xlsx, the chunk of code treats data from the same spreadsheet as the previous chunk.\nThe packages I use are all imported in the chunk immediately above this one. Longer comments, or comments that I think contain information that may be relevant to the reader is commented out like here, whereas\ncomments that are mainly for the sake of structure and do not contain new or noteworthy information are just hashtagged out.\nAs of 20/2/25 this script is unrevised by others. \nThe script runs reasonably efficiently with a few exceptions, it does however rely quite heavily on the pandas-package atm.\nIn terms of robustness, I rely quite heavily

### Some dictionaries

In [7]:
'''dictionaries for mapping from raw data onto GR-set members'''

dict_e={'liq_biofuel':'Biooil','other_oil':'Oil products','coal':'Coal and coke','firewood':'Firewood and woodchips','refin_gas':'Refinery gas','semi_refin_oil':'Semi-refined oil','straw':'Straw for energy purposes','crude':'Crude oil','bunk_planes':'Bunkering of Danish operated planes on foreign territory','bunk_ships':'Bunkering of Danish operated vessels on foreign territory','bunk_trucks':'Bunkering of Danish operated trucks on foreign territory','jet_petro':'Jet petroleum','diesel_transp':'Diesel for transport','gasol_transp':'Gasoline for transport','heat_pump':'Heat pumps','renewable':'Renewable energy','waste_oil':'Waste oil','wood_waste':'Wood waste','wood_pellets':'Wood pellets','district_heat':'District heat','natgas':'Natural gas incl. biongas','natgas_extract':'Natural gas (Extraction)'}

dict_transaction={'transmis_loss':'transmission_losses','cons_inter':'input_in_production','cons_hh':'household_consumption','import':'imports','invent_change':'inventory'}

dict_ebalitems={'ws_marg':'EAV','ret_marg':'DAV','basic':'BASE','co2_xbio':'co2ubio','co2_eq':'co2e','co2_bio':'co2bio'}

dict_a={'tax_products':'TaxSub','tax_vat':'Moms','emp_comp':'SalEmpl','subs_other_production':'OthSubs','tax_other_production':'OthTax','gross_surplus':'OvProd'} #turister

### energy_and_emissions.xlsx

### Emissions

In [ ]:
energy_and_emissions=pd.read_excel(r'data\energy_and_emissions.xlsx',keep_default_na=True)

'''rename coslumns for compatibility with data loading'''
energy_and_emissions.rename(columns={'indu':'d','product':'e','purp':'es','flow':'transaction'},inplace=True)
energy_and_emissions.set_index(['year','bal','transaction','d','es','e'],drop=True,inplace=True)

energy_and_emissions=energy_and_emissions.stack().to_frame(name='level')
energy_and_emissions.index.set_names(['year','bal','transaction','d','es','e','ebalitems'],inplace=True)
energy_and_emissions.reset_index(inplace=True)
energy_and_emissions['es'].fillna('unspecified',inplace=True)

energy_and_emissions['transaction'].replace(dict_transaction,inplace=True)
energy_and_emissions['ebalitems'].replace(dict_ebalitems,inplace=True)

'''replace those pesky nans with a target-specific label'''
energy_and_emissions['d'] = energy_and_emissions.apply(lambda row: 'xOth' if pd.isna(row['d']) and row['transaction'] == 'export' 
                               else ('invt' if pd.isna(row['d']) and row['transaction'] == 'inventory' 
                                     else ('tl' if pd.isna(row['d']) and row['transaction'] == 'transmission_losses' 
                                           else ('natural_input' if pd.isna(row['d']) and row['transaction'] == 'nat_input'
                                                 else ('residual' if pd.isna(row['d']) and row['transaction'] == 'res_input'
                                                       else ('19000' if pd.isna(row['d']) and row['transaction']=='imports'
                                                            else row['d']))))), axis=1)

energy_and_emissions['e']=energy_and_emissions['e'].replace({'liq_biofuel':'Biooil','other_oil':'Oil products','coal':'Coal and coke','firewood':'Firewood and woodchips','refin_gas':'Refinery gas','semi_refin_oil':'Semi-refined oil','straw':'Straw for energy purposes','crude':'Crude oil','bunk_planes':'Bunkering of Danish operated planes on foreign territory','bunk_ships':'Bunkering of Danish operated vessels on foreign territory','bunk_trucks':'Bunkering of Danish operated trucks on foreign territory','jet_petro':'Jet petroleum','diesel_transp':'Diesel for transport','gasol_transp':'Gasoline for transport','heat_pump':'Heat pumps','renewable':'Renewable energy','waste_oil':'Waste oil','wood_waste':'Wood waste','wood_pellets':'Wood pellets','district_heat':'District heat','natgas':'Natural gas incl. biongas','natgas_extract':'Natural gas (Extraction)'})
'''Bioethanol og biodiesel mangler'''

'''add t=2019'''
#energy_and_emissions=pd.concat([energy_and_emissions,pd.DataFrame(columns=['t'],index=energy_and_emissions.index,data=2020)],axis=1)


# set column order
energy_and_emissions = energy_and_emissions[['ebalitems', 'transaction', 'd', 'es', 'e', 'year', 'level']]

'''build sets'''
transaction=gp.Set(m,name='transaction',description='HEY. set of transaction types')
'''We add 'energy' manually to d, because it does not appear in data,
and in order to define equations in the model on the demand component 'energy', we need to first declare that it
is in fact a set member. '''
#d=gp.Set(m,'d',description='demand components',records='energy')
d=gp.Set(m,'d',description='HEY. demand components')
es=gp.Set(m,'es',description='HEY.energy service')
e=gp.Set(m,'e',description='HEY. energy products by industry')
ebalitems=gp.Set(m,'ebalitems',description='HEY. identifiers tax joules prices etc for energy components by demand components')
t=gp.Set(m,'t',description='HEY. year')

EnergyBalance=gp.Parameter(m,'EnergyBalance',domain=[ebalitems,transaction,d,es,e,t],description='Main data input with regards to energy and energy-related emissions',records=energy_and_emissions.values.tolist(),domain_forwarding=True)

demand_transaction=gp.Set(m,name='demand_transaction',domain=[transaction],description='hej',records=['production','input_in_production','export','inventory','transmission_losses'])
em=gp.Set(m,name='em',domain=[ebalitems],description='HEY. emission types',records=['ch4','co2ubio','n2o','co2e','co2bio'])
etaxes=gp.Set(m,name='etaxes',domain=[ebalitems],description='HEY. taxes from ebalitems',records=['co2_tax','so2_tax','nox_tax','ener_tax','pso_tax'])


C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\830845114.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  energy_and_emissions['es'].fillna('unspecified',inplace=True)
C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\830845114.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

### Nonenergyemissions

In [9]:
non_energy_emissions=pd.read_excel(r'data\non_energy_emissions.xlsx',keep_default_na=True)
non_energy_emissions.set_index(['bal','flow','indu'],inplace=True)
non_energy_emissions.drop(columns=['year'],inplace=True) #annoying manual bs, will fix later
non_energy_emissions=non_energy_emissions.stack().to_frame(name='level')
non_energy_emissions.dropna(inplace=True)
non_energy_emissions.index.rename(['bal','transaction','d','ebalitems'],inplace=True)
non_energy_emissions.reset_index(inplace=True)
non_energy_emissions.drop(columns='bal',inplace=True)

non_energy_emissions['transaction'].replace(dict_transaction,inplace=True)
non_energy_emissions['ebalitems'].replace(dict_ebalitems,inplace=True)
'''
t-column
'''
non_energy_emissions_t=pd.concat([non_energy_emissions,pd.DataFrame(columns=['t'],index=non_energy_emissions.index,data=2020)],axis=1)


# set column order
non_energy_emissions_t = non_energy_emissions_t[['ebalitems', 'transaction', 'd', 't', 'level']]

non_energy_emissions_t=gp.Parameter(m,name='NonEnergyEmissions',domain=[ebalitems,transaction,d,t],description='HEY. emission from consumption of non-energy',records=non_energy_emissions_t.values.tolist(),domain_forwarding=True)

C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\2773027590.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  non_energy_emissions['transaction'].replace(dict_transaction,inplace=True)
C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\2773027590.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

### IO-loading

In [10]:
'''gampy does not support domain forwarding from set to set.
To construct a superset of IO-sectors we must initially populate the superset, then provide it as domain for subsets, which we can then populate using the tried and tested domain_forwarding method.
To populate the superset, we must load data from the io-spreadsheets.
On the data:
The Danish IO-tables are received in long-format and matrix-format. I read the long-format for convenience, but the content of the data is better understood when veiwing in matrix-format.
The columns consist of demand-components, which can be either sectors of the economy, export, investment or some final-demand component such as food or housing.
The rows consist of inputs, and can be subdivided into three categories, domestic production, import and primary inputs.
The primary inputs are taxes, subsidies, employee compensation and the likes.
Imported and domestically produced inputs are industry-outputs such that a number in any entry of the IO-matrix can be read as the supply from the row-index to the column-index.
To make this data compatible with the model, we must first inform the model how to interpret the indices. We do this by defining sets.
For instance the set "i" is the set of sectors in the model, we can therefore say that the rows of the import- and domestically produced part of the io-table, consist of members of i. 
The set d is the set of demand-components and corresponds to the columns of the io-tables.
By creating separate variables for imported and domestically produced supply from the io-table, we can define a variable like vIO_y, and define in on [i,d,t], which we can then read as 
domestic sector i's supply to demand component d at year t. Atm, I think the easiest to follow in the extremely likely event that data is not in the exact same format, is the current version
in which I explicitly label columns in the dataframes according to their GR-set-counterparts. 
It is important not to mess with the sets, as they are called explicitly in the model, always by name and on occasion we also refer to specific elements.
The consequence of this is that names have to be inserted here. This will almost certainly need to be edited for other sources of data.
I try to make apparent from the naming throughout where columns and rows are meant to end up in the model which
I suspect will ultimately be more convenient for adapting this code to other datasets than attempting to accomplish the
highest possible degree of automization.
'''
#energy
io_energy=pd.read_excel(r'data\io_energy_long_format.xlsx',keep_default_na=True)
io_energy_forlater=io_energy.copy(deep=True)
io_energy.rename(columns={'row_l2':'i','col_l1':'DELETE','col_l2':'d','value':'level'},inplace=True)

#regular
io=pd.read_excel(r'data\io_long_format.xlsx')
io_forlater=io.copy(deep=True)
io.rename(columns={'row_l2':'i','col_l1':'DELETE','col_l2':'d','value':'level'},inplace=True)

'''fill nans based on previous entries. In Danish dataset, col_l1 contains supercategories for col_l2.
Meanwhile, we use col_l2, which contains useful categories such as sectors (as opposed to cons_inter). 
Categories which do not have a subcategory, such as cons_publ (public consumption) have an empty cell at col_l2.
In this program I create the set of demand components d from its members, so in order to represent the demand components that do
not have multiple subcategories, I run the lines below which fills the appropriate GR-demand component based on the supercategory, i.e.
if row10 supercategory is export, replace NaN in the demand component column of row10 with xOth.
'''
io['d'] = io.apply(lambda row: 'xOth' if pd.isna(row['d']) and row['DELETE'] == 'export' 
                               else ('invt' if pd.isna(row['d']) and row['DELETE'] == 'invent_change' 
                                     else ('g' if pd.isna(row['d']) and row['DELETE'] == 'cons_publ' 
                                           else ('iB' if pd.isna(row['d']) and row['DELETE'] == 'invest_build'
                                                 else ('iT' if pd.isna(row['d']) and row['DELETE'] == 'invest_trans'
                                                       else ('iM' if pd.isna(row['d']) and row['DELETE']=='invest_other'
                                                            else row['d']))))), axis=1)

io_energy['d'] = io_energy.apply(lambda row: 'xOth' if pd.isna(row['d']) and row['DELETE'] == 'export' 
                               else ('invt' if pd.isna(row['d']) and row['DELETE'] == 'invent_change' 
                                     else ('g' if pd.isna(row['d']) and row['DELETE'] == 'cons_publ' 
                                           else ('iB' if pd.isna(row['d']) and row['DELETE'] == 'invest_build'
                                                 else ('iT' if pd.isna(row['d']) and row['DELETE'] == 'invest_trans'
                                                       else ('iM' if pd.isna(row['d']) and row['DELETE']=='invest_other'
                                                            else row['d']))))), axis=1)

'''Editors note: At the time of writing, the model is not equipped to handle the distinction between con_hh and cons_hh_foreign, therefore
for the time being we simply add them together, I do this by creating a boolean mask to identify rows where the aforementioned supercategory-column 
contains the string cons_hh, which in the GR dataset is both cons_hh and cons_hh_foreign_tou. I then build a dataframe of the entries with only the members of
the original frame(s) that has has cons_hh or cons_hh_foreign_tou as supercategory. I then group entries in this dataset based on entries in the other columns (except of course level) and add them together.
The rows that were selected for this process is then dropped from the original frame(s) and the aggregated rows are added back.
'''
mask=io["DELETE"].str.contains("cons_hh")
mask_ene = io_energy["DELETE"].str.contains("cons_hh")
io_agg = io[mask].groupby(io.columns.difference(["DELETE","level"]).tolist(), as_index=False)["level"].sum()
io_energy_agg= io_energy[mask_ene].groupby(io_energy.columns.difference(["DELETE","level"]).tolist(), as_index=False)["level"].sum()
'''drop rows that had been aggregated'''
io = io[~mask]
io_energy=io_energy[~mask_ene]
'''drop DELETE'''
io.drop(columns=['DELETE'],inplace=True)
io_energy.drop(columns=['DELETE'],inplace=True)
'''reconcat'''
io=pd.concat([io,io_agg])
io_energy=pd.concat([io_energy,io_energy_agg])
'''add t's'''
io=pd.concat([io,pd.DataFrame(columns=['t'],index=io.index,data=2020)],axis=1)
io_energy=pd.concat([io_energy,pd.DataFrame(columns=['t'],index=io_energy.index,data=2020)],axis=1)
'''reorder for consistency with GR-variables'''
io=io[['row_l1','i', 'd', 't', 'level']]
io_energy=io_energy[['row_l1','i', 'd', 't', 'level']]
'''disentagle'''
io_y=io[io['row_l1']=='production']
io_m=io[io['row_l1']=='import']
io_a=io[io['row_l1']=='prim_input']
io_ene_y=io_energy[io_energy['row_l1']=='production']
io_ene_m=io_energy[io_energy['row_l1']=='import']
io_ene_a=io_energy[io_energy['row_l1']=='prim_input']
'''drop columns'''
io_y.drop(columns=['row_l1'],inplace=True)
io_m.drop(columns=['row_l1'],inplace=True)
io_ene_a.drop(columns=['row_l1'],inplace=True)
io_ene_y.drop(columns=['row_l1'],inplace=True)
io_ene_m.drop(columns=['row_l1'],inplace=True)
io_a.drop(columns=['row_l1'],inplace=True)
'''apply a_dict'''
io_a['i'].replace(dict_a,inplace=True)
io_ene_a['i'].replace(dict_a,inplace=True)
io_a=io_a.groupby(['d','i','t'],as_index=False).agg({'level':'sum'})
io_combined_a = pd.concat([io_a, io_ene_a]).groupby(io_a.columns.difference(["level"]).tolist(), as_index=False)["level"].sum()
'''add energy to non-energy for vIO_{y,m,a}'''
io_combined_y = pd.concat([io_y, io_ene_y]).groupby(io_y.columns.difference(["level"]).tolist(), as_index=False)["level"].sum()
io_combined_m=pd.concat([io_m, io_ene_m]).groupby(io_m.columns.difference(["level"]).tolist(), as_index=False)["level"].sum()
io_combined_a=io_combined_a.groupby(['d','i','t'],as_index=False).agg({'level':'sum'})
'''initialize sets'''
i=gp.Set(m,name='i',description='sectors',domain_forwarding=True)
a_rows_=gp.Set(m,'a_rows_',description='other rows in the input-output table',domain_forwarding=True)
'''add tot'''
io_y_agg=io_y.groupby('d',as_index=False)['level'].sum()
io_y_agg.insert(0,'i','tot')
io_y_agg.insert(2,'t','2020')
io_y=pd.concat([io_y,io_y_agg])
io_m_agg=io_m.groupby('d',as_index=False)['level'].sum()
io_m_agg.insert(0,'i','tot')
io_m_agg.insert(2,'t','2020')
io_m=pd.concat([io_m,io_m_agg])
io_a_agg=io_a.groupby('d',as_index=False)['level'].sum()
io_a_agg.insert(0,'i','tot')
io_a_agg.insert(2,'t','2020')
io_a=pd.concat([io_a,io_a_agg])
io_combined_y_agg=io_combined_y.groupby('d',as_index=False)['level'].sum()
io_combined_y_agg.insert(0,'i','tot')
io_combined_y_agg.insert(2,'t','2020')
io_combined_y=pd.concat([io_combined_y,io_combined_y_agg])
io_combined_m_agg=io_combined_m.groupby('d',as_index=False)['level'].sum()
io_combined_m_agg.insert(0,'i','tot')
io_combined_m_agg.insert(2,'t','2020')
io_combined_m=pd.concat([io_combined_m,io_combined_m_agg])
'''change order for GR-compatibility'''
io_a=io_a[['i','d','t','level']]
io_combined_a=io_combined_a[['i','d','t','level']]

#export
vIOxE_y=gp.Parameter(m,name='vIOxE_y',domain=[i,d,t],description='HEY. non-energy IO of domestic production',records=io_y.values.tolist(),domain_forwarding=True)
vIOxE_m=gp.Parameter(m,name='vIOxE_m',domain=[i,d,t],description='HEY. non-energy IO of imports',records=io_m.values.tolist(),domain_forwarding=True)

vIO_y=gp.Parameter(m,name='vIO_y',domain=[d,i,t],description='.HEY Production IO',records=io_combined_y.values.tolist(),domain_forwarding=True)
vIO_m=gp.Parameter(m,name='vIO_m',domain=[d,i,t],description='HEY. Production IO',records=io_combined_m.values.tolist(),domain_forwarding=True)

vIO_a=gp.Parameter(m,name='vIO_a',domain=[a_rows_,d,t],description='HEY. other IO',records=io_combined_a.values.tolist(),domain_forwarding=True)
vIOxE_a=gp.Parameter(m,name='vIOxE_a',domain=[a_rows_,d,t],description='HEY. non energy other IO',records=io_a.values.tolist(),domain_forwarding=True)

'''IO investment from here'''

io_inv=pd.read_excel(r'data\io_invest_long_format.xlsx',keep_default_na=True)
io_inv.rename(columns={'col':'i','invest_group':'k','value':'level'},inplace=True)
io_inv_dict={'invest_build':'iB','invest_other':'iM','invest_trans':'iT'}
'''apply dict for GR-compatible codes'''
io_inv['k']=io_inv['k'].replace(io_inv_dict)

'''Define set for capital types'''
k=gp.Set(m,name="k",description='HEY. capital types',domain_forwarding=True)

'''atm we do not care abt. "sender" of capital, just building qI_k_i'''
io_inv_qI_k_i=io_inv.copy(deep=True)
io_inv_qI_k_i.drop(columns=['row_l1','row_l2'],inplace=True)
io_inv_qI_k_i=pd.concat([io_inv_qI_k_i,pd.DataFrame(columns=(['t']),index=io_inv_qI_k_i.index,data=2020)],axis=1)
io_inv_qI_k_i=io_inv_qI_k_i[['k','i','t','level']]
'''aggregate'''
io_inv_qI_k_i_agg=io_inv_qI_k_i.groupby(['k','i','t'],as_index=False).agg({'level':'sum'})
#send it
'''Note: There are many sectors here that are not represented in the other IO-tables, should be brought up to date'''
#qI_k_i=gp.Parameter(m,'qI_k_i',domain=[k,i,t],description='hEY. Real capital stock by capital type and industry',records=io_inv_qI_k_i_agg.values.tolist(),domain_forwarding=True)

#i_=gp.Alias(m,name='i_',alias_with=i)




C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\2154421953.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  io_y.drop(columns=['row_l1'],inplace=True)
C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\2154421953.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  io_m.drop(columns=['row_l1'],inplace=True)
C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\2154421953.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  io_en

'Note: There are many sectors here that are not represented in the other IO-tables, should be brought up to date'

### demand-side based io incl. employee compensation

In [11]:

'''build qL'''
io_qRxE=io_forlater[io_forlater['col_l1']=='cons_inter']
io_qRxE.rename(columns={'col_l2':'i','value':'level'},inplace=True)

io_qRxE['row_l2']=io_qRxE['row_l2'].replace(dict_a)

io_l=io_qRxE[io_qRxE['row_l2']=='SalEmpl']
io_l_s=io_l.groupby(['i','year'],as_index=False).agg({'level':'sum'})
io_l_s=gp.Parameter(m,name='qL',domain=[i,t],description='HEY. Wage expenses',records=io_l_s.values.tolist(),domain_forwarding=True)
'''nemployed from here'''
employed=pd.read_excel(r'data\employed.xlsx',keep_default_na=True)
nemployed_frame = pd.DataFrame(columns=['t', 'level'], data=[[2020, employed['employed'].sum()]])
nemployed=gp.Parameter(m,name='nEmployed',domain=[t],description='HEY. Total number of employees: Mangler selvstændige',records=nemployed_frame.values.tolist())

C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\1597611272.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  io_qRxE.rename(columns={'col_l2':'i','value':'level'},inplace=True)
C:\Users\B247549\AppData\Local\Temp\ipykernel_12948\1597611272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  io_qRxE['row_l2']=io_qRxE['row_l2'].replace(dict_a)


### Prepare for capital transfer

In [12]:
fixed_assets=pd.read_excel(r'data\fixed_assets.xlsx',keep_default_na=True)

m.write('dataa_ny.gdx')